<a href="https://colab.research.google.com/github/drwpls/cuda/blob/HW3/HW3/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Họ tên: Phan Lộc Sơn

MSSV: 19120033

# HW3: Các loại bộ nhớ trong CUDA

Với các GPU tương đối mới thì để biên dịch chỉ cần dùng câu lệnh: \
`nvcc tên-file.cu -o tên-file-chạy`

Nhưng trên Colab mình thường lấy được GPU khá cũ là Tesla K80 với compute capability (phiên bản phần cứng) là 3.7; để biên dịch đúng với GPU khá cũ này thì bạn cần dùng câu lệnh: \
`nvcc -arch=sm_37 tên-file.cu -o tên-file-chạy` \
Trong đó, 37 chính là compute capability của GPU Tesla K80.

Để phòng trường hợp khi làm bài bạn lấy được GPU có compute capability x.x nhưng khi chấm bài Thầy lại lấy được GPU có compute capability khác x.x, dưới đây mình sẽ có đoạn code Python để tự động lấy 2 con số ứng với compute capability của GPU và lưu vào 2 biến `major` và `minor`:


In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


Một khi đã chạy đoạn code Python ở trên, để biên dịch thì bạn sẽ dùng câu lệnh: \
`nvcc -arch=sm_{major}{minor} tên-file.cu -o tên-file-chạy`

Dưới đây, khi làm bài thì bạn có thể tùy ý thêm/xóa cell. Đừng xóa mấy cell có chữ của Thầy là được.

Fetch code from github

In [2]:
%cd /content
!rm -rf cuda
!git clone -b HW3 https://github.com/drwpls/cuda.git

/content
Cloning into 'cuda'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 114 (delta 41), reused 52 (delta 11), pack-reused 0
Receiving objects: 100% (114/114), 1.77 MiB | 11.09 MiB/s, done.
Resolving deltas: 100% (41/41), done.


Change working directory to HW3 folder:

In [3]:
%cd ./cuda/HW3

/content/cuda/HW3


You can verify by run `pwd`

In [4]:
!pwd

/content/cuda/HW3


Compile HW3 binary

In [5]:
!nvcc -arch=sm_{major}{minor} HW3.cu -o HW3 

Run the binary

In [6]:
!./HW3 in.pnm out.pnm 16 16

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x16, grid size 32x32
Kernel time: 0.468992 ms
Error: 0.000703

Kernel 2, block size 16x16, grid size 32x32
Kernel time: 0.305920 ms
Error: 0.000703

Kernel 3, block size 16x16, grid size 32x32
Kernel time: 0.233504 ms
Error: 0.000703


In [7]:
!./HW3 in.pnm out.pnm 32 32

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15843721216 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 32x32, grid size 16x16
Kernel time: 0.346112 ms
Error: 0.000703

Kernel 2, block size 32x32, grid size 16x16
Kernel time: 0.339968 ms
Error: 0.000703

Kernel 3, block size 32x32, grid size 16x16
Kernel time: 0.262304 ms
Error: 0.000703


Nhận xét: Cơ bản, hàm Kernel 3 chạy nhanh hơn Kernel 2 và Kernel 2 chạy nhanh hơn Kernel 1.

- Hàm kernel 2 chạy nhanh hơn kernel 1 vì: 
 + Dùng shared memory để load vào trước rồi mới tính toán. Thay vì mỗi lần tinh là một lần load data từ global memory vao thì kernel2 load vào shared_memory hết rồi mới tính.
 + Khi block size = 16 x 16, hàm kernel 2 nhanh hơn hẳn kernel 1 vì: Trong 1 block, có ít threads hơn -> khi các threads chạy và load từ global mem sang shared mem, ít memory transaction bị bank conflict hơn -> tận dụng shared memory tố hơn.

- Hàm kernel 3 chạy hơn hơn kernel 2 vì nó load filter data từ bộ nhớ constant nằm ở layer thấp hơn so với load trực tiêp từ gloabal memory (mỗi threads ở kernel 2 và kernel 1 phải load toàn bộ filter vào: filterWidth^2 phần tử)